## Fake News Classifier Using LSTM

**Competition Link :** https://www.kaggle.com/c/fake-news/overview

In [63]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100% !important; }</style>"))
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 22, 7
import warnings
warnings.filterwarnings('ignore')

In [103]:
import pandas as pd 
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\92304\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
df = pd.read_csv('./data/train.csv')
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [66]:
df.dropna(inplace=True)

In [67]:
X = df.drop('label', axis=1)
y = df['label']
X.shape, y.shape

((18285, 4), (18285,))

In [68]:
vocab_size = 5000

### One Hot Representation

In [69]:
messages = X.copy()
messages.reset_index(inplace=True)

In [70]:
## Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower().split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [71]:
corpus[0:3]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire']

In [72]:
onehot_repr = [one_hot(word, vocab_size) for word in corpus]
onehot_repr

[[2745, 1869, 4340, 2540, 4938, 2617, 3469, 1955, 2237, 2913],
 [2836, 4718, 236, 1166, 4827, 3970, 2869],
 [4630, 1158, 4642, 1832],
 [4771, 3587, 3843, 1202, 3114, 806],
 [1663, 4827, 2688, 69, 3596, 2871, 4827, 1605, 2987, 1373],
 [4941,
  3837,
  2323,
  1420,
  2256,
  3091,
  2821,
  452,
  4403,
  1408,
  3072,
  492,
  4732,
  200,
  2869],
 [1375, 861, 4970, 4465, 3860, 1183, 3658, 4937, 2112, 1819, 564],
 [4408, 2434, 3317, 488, 66, 378, 3091, 1105, 2112, 1819, 564],
 [3526, 2236, 2216, 990, 409, 2067, 1697, 4860, 3091, 4163],
 [3379, 272, 1050, 1644, 2238, 4553, 3518, 765],
 [997, 979, 3224, 4786, 2503, 2954, 1423, 3223, 316, 1833, 2661],
 [1202, 2343, 4938, 2067, 3091, 66],
 [1010, 1561, 1670, 4652, 3856, 3358, 39, 1680, 1455],
 [1604, 3927, 1221, 3014, 2032, 2094, 936, 2112, 1819, 564],
 [4677, 713, 4424, 1620, 1094, 2112, 1819, 564],
 [1772, 4358, 3276, 1217, 1634, 2128, 10, 1924, 785, 4274],
 [1821, 4344, 4718],
 [4368, 3771, 2718, 4060, 3091, 4771, 2093, 2869],
 [4730, 

### Embedding Representaion 

In [73]:
sent_len = 20
embadded_docs = pad_sequences(onehot_repr, padding='pre', maxlen= sent_len)
print(embadded_docs)
print(embadded_docs[0])

[[   0    0    0 ... 1955 2237 2913]
 [   0    0    0 ... 4827 3970 2869]
 [   0    0    0 ... 1158 4642 1832]
 ...
 [   0    0    0 ... 2112 1819  564]
 [   0    0    0 ... 3808 4705 3058]
 [   0    0    0 ... 2904 2133 2281]]
[   0    0    0    0    0    0    0    0    0    0 2745 1869 4340 2540
 4938 2617 3469 1955 2237 2913]


In [104]:
## Creating The Model 
embedding_vector_features = 40 
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length= sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [75]:
len(embadded_docs), y.shape

(18285, (18285,))

In [76]:
## Coverting the input features to array
X_final = np.array(embadded_docs)
y_final = np.array(y)
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [105]:
X_train, X_test , y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((12250, 20), (6035, 20), (12250,), (6035,))

In [106]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size= 64)

Epoch 1/20
192/192 [==============================] - 5s 16ms/step - loss: 0.3616 - accuracy: 0.8288 - val_loss: 0.2006 - val_accuracy: 0.9150
Epoch 2/20
192/192 [==============================] - 3s 15ms/step - loss: 0.1521 - accuracy: 0.9397 - val_loss: 0.1886 - val_accuracy: 0.9229
Epoch 3/20
192/192 [==============================] - 3s 14ms/step - loss: 0.1118 - accuracy: 0.9588 - val_loss: 0.2064 - val_accuracy: 0.9171
Epoch 4/20
192/192 [==============================] - 3s 14ms/step - loss: 0.0831 - accuracy: 0.9696 - val_loss: 0.2232 - val_accuracy: 0.9099
Epoch 5/20
192/192 [==============================] - 3s 15ms/step - loss: 0.0639 - accuracy: 0.9775 - val_loss: 0.2805 - val_accuracy: 0.9074
Epoch 6/20
192/192 [==============================] - 3s 15ms/step - loss: 0.0443 - accuracy: 0.9851 - val_loss: 0.2945 - val_accuracy: 0.9135
Epoch 7/20
192/192 [==============================] - 4s 19ms/step - loss: 0.0354 - accuracy: 0.9879 - val_loss: 0.3466 - val_accuracy: 0.9104

## Performance Metrics and Accuracy

In [107]:
print(classification_report(y_test, model.predict(X_test)>0.5))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.91      0.90      0.90      6035

